In [1]:
# notebook to illustrate document similarity clusters

In [1]:
# following only used for development, reloads the modules with any code changes
%load_ext autoreload
%autoreload 2

# inline matplotlib charts
%matplotlib inline

In [2]:
# import our text mining toolkit
import text_mining_toolkit as tmt

In [3]:
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/recipes/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(directory_of_files="data_sets/mystery_corpus_01/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/iraq_inquiry/txt/", text_filename_pattern="the-report*.txt")
cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/clinton_emails/txt/", text_filename_pattern="C0*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/shakespeare_macbeth/txt/", text_filename_pattern="macbeth_act_0?_scene_0?.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/mixed/txt/", text_filename_pattern="*.txt")

content_directory =  data_sets/clinton_emails/txt/
text_filename_pattern =  C0*.txt
self.documents populated =  7945


In [27]:
# first clear index
tmt.index_wordcount.delete_index(cr.content_directory)

# for all documents in corpus
for document_name in cr.get_documents():
    print("processing ", document_name)

    # get document text
    document_text = cr.get_text_by_document(document_name)

    # simplify whitespace (remove newlines)
    b = tmt.text_processing.simplify_whitespace(document_text)

    # only keep alphanumeric characters, removes punctuation
    c = tmt.text_processing.keep_only_alphanumeric(b)

    # make lowercase
    d = tmt.text_processing.to_lowercase(c)

    # split into words list
    dl = tmt.text_processing.split_text_into_words(d)
    
    # build n-grams
    #gl = tmt.word_processing.build_ngrams_from_words(dl,2)

    # remove stop words
    #el = tmt.word_processing.remove_stop_words(dl, "./stopwords/minimal-stop.txt")
    
    # update index
    tmt.index_wordcount.create_wordcount_index_for_document(cr.content_directory, document_name, dl)
    pass



processing  01.txt
processing  02.txt
processing  03.txt
processing  04.txt
processing  05.txt
processing  06.txt
processing  07.txt
processing  08.txt
processing  09.txt
processing  10.txt
processing  11.txt
processing  12.txt
processing  13.txt
processing  C05739545.txt
processing  C05739546.txt
processing  C05739547.txt
processing  C05739550.txt
processing  C05739554.txt
processing  C05739559.txt
processing  C05739560.txt
processing  C05739561.txt
processing  C05739562.txt
processing  C05739563.txt
processing  C05739564.txt
processing  C05739565.txt
processing  C05739567.txt
processing  macbeth_act_01_scene_01.txt
processing  macbeth_act_01_scene_02.txt
processing  macbeth_act_01_scene_03.txt
processing  macbeth_act_01_scene_04.txt
processing  macbeth_act_01_scene_05.txt
processing  macbeth_act_01_scene_06.txt
processing  macbeth_act_01_scene_07.txt
processing  macbeth_act_02_scene_01.txt
processing  macbeth_act_02_scene_02.txt
processing  macbeth_act_02_scene_03.txt
processing  mac

In [28]:
# merge document indices into a corpus index
tmt.index_wordcount.merge_wordcount_indices_for_corpus(cr.content_directory)

saving corpus word count index ...  data_sets/mixed/txt/index_wordcount.hdf5


In [33]:
# delete and build relevance index

tmt.index_relevance.delete_index(cr.content_directory)
tmt.index_relevance.calculate_relevance_index(cr.content_directory)

removed relevance index file:  data_sets/mixed/txt/index_relevance.hdf5
saving corpus relevance index ...  data_sets/mixed/txt/index_relevance.hdf5


In [8]:
# calculate document similarities

# delete if already exists
tmt.index_doc_similarity.delete_matrix(cr.content_directory)

# calclate doc1-doc2 similarity
%time tmt.index_doc_similarity.create_doc_similarity_matrix(cr.content_directory)

*** warning ** : invalid value encountered in double_scalars
doc1 C05739545.txt
sum =  0.43282165670656747
                        0.0
0                       0.0
00                      0.0
000                     0.0
0000                    0.0
00000000000000          0.0
000001s2housegov        0.0
00000e2511c8printyes    0.0
000014                  0.0
0000779fd2achtmly       0.0
000077qfd2achtml        0.0
0001                    0.0
000151                  0.0
000154                  0.0
000325                  0.0
000336                  0.0
00041vahoocom           0.0
000508                  0.0
000608                  0.0
000845                  0.0
000907                  0.0
001                     0.0
001011                  0.0
001057                  0.0
001103                  0.0
001122                  0.0
001138                  0.0
001145                  0.0
001236                  0.0
001319                  0.0
                       ... 
zsbwhoeop               0

In [17]:
#tmt.index_doc_similarity.print_matrix(cr.content_directory)

In [18]:
# get document similarities order by value
doc1_doc2_similarity = tmt.index_doc_similarity.get_doc_pairs_by_similarity(cr.content_directory)
doc1_doc2_similarity[:20]

,doc1,doc2,similarity
0,17.txt,04.txt,1.000000
1,06.txt,05.txt,0.858670
2,11.txt,10.txt,0.818525
3,13.txt,07.txt,0.745713
4,11.txt,09.txt,0.700727
5,21.txt,20.txt,0.653643
6,20.txt,17.txt,0.646402
7,17.txt,12.txt,0.645868
8,21.txt,03.txt,0.632884
9,04.txt,03.txt,0.627740


In [49]:
# visualise the doc similarities
tmt.visualisation.plot_force_directed_graph(doc1_doc2_similarity[:200])

force-directed graph